In [59]:
!pip install nltk gensim openai

In [60]:
import pandas as pd
import time
import numpy as np
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from gensim.parsing.preprocessing import STOPWORDS, strip_tags, strip_numeric, strip_punctuation, strip_multiple_whitespaces, remove_stopwords
from groq import Groq
import os
from dotenv import load_dotenv
from nltk.tokenize import sent_tokenize
import json

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/oskarroeske/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# EDA

In [80]:
df_reports = pd.read_csv("preprocessed_reports.csv")
df_paragraphs = pd.read_csv("preprocessed_paragraphs.csv")

In [81]:
df_paragraphs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10571 entries, 0 to 10570
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    10571 non-null  int64 
 1   document_id   10571 non-null  int64 
 2   paragraph_id  10571 non-null  int64 
 3   paragraph     10571 non-null  object
dtypes: int64(3), object(1)
memory usage: 330.5+ KB


In [82]:
def clean_text_again(text):
    text = strip_tags(text)
    text = strip_multiple_whitespaces(text)  # Normalize whitespaces
    return text

In [83]:
df_paragraphs["paragraph"] = df_paragraphs["paragraph"].apply(clean_text_again)

# FINBERT

# KeyBERT

# LLM Argument Mining

- Provide List of pre-defined arguments

In [84]:
list_of_arguments = [
    "Earnings per Share",
    "Cash Flow",
    "Revenue",
    "Return On Equity",
    "Margins",
    "Cost Management",
    "Dividend Policy",
    "Investments",
    "Balance Sheet",
    "Long-term Growth",
    "Merges and Acquisition",
    "Refranchising",
    "Sustainability",
    "Employees",
    "Research and Development",
    "Marketing",
    "Shares Repurchase",
    "Processes",
    "Leadership",
    "Innovation",
    "Product Characteristics",
    "Pricing Strategy",
    "Production",
    "Technology Trends",
    "Market Share",
    "Market Expansion",
    "Competitors",
    "Global Presence",
    "Industry Outlook",
    "Regulations",
    "Partnerships and Collaborations",
    "Supply Chain",
    "Economic Conditions",
    "Demand",
    "Customer"
]


In [ ]:
from openai import OpenAI
import json

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI(api_key=api_key)

def find_arguments(text):

    categories_string = ', '.join(f'"{cat}"' for cat in list_of_arguments)

    system_content = f"""You will classify financial sentences into predefined categories or create a new one if needed:
- Assign arguments to the best-matching category. Create a new category only if no match fits.
- IGNORE brand names/Target Prices/Recommendations/Valuation as arguments.
- If no meaningful argument is mentioned in the text, return "None"
- Include new categories in the output.
- Return only the best-fitting category as a String
- Print only the result! No other text!
Categories: [{categories_string}]
"""

    user_content = f"""
    text: {text}
    """
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system", 
                "content":[
                    {
                        "type":"text",
                        "text":system_content
                    }
                ] 
            },
            {
                "role": "user", 
                "content":[
                    {
                        "type":"text",
                        "text":user_content
                    }
                ] 
            }
        ],
        response_format={
        "type": "json_object"
        }
    )
    return response.choices[0].message.content

In [85]:
df_paragraphs = df_paragraphs[0:3]

In [86]:
df_paragraphs

,Unnamed: 0,document_id,paragraph_id,paragraph
0,0,1,1,: $150.00 We raise our estimates for 3Q16 as w...
1,1,1,2,Our $150 target price embeds a 10-year OIBDA g...
2,2,1,3,"FB is the largest social network, with 1.65B u..."


In [87]:
def extract_arguments_and_sentiment(commentary):
    global list_of_arguments  # Ensure we can modify the global categories list
    
    # Call the find_arguments function with the selected API key
    result_json = find_arguments(commentary)   
    
    try:
        # Parse the JSON response
        result_dict = json.loads(result_json)
        
        # Check for new categories and add them to the global list
        for category in result_dict.keys():
            if category not in list_of_arguments:
                list_of_arguments.append(category)  # Dynamically update the categories list
        
        time.sleep(10)
        return result_dict  # Return the result
    except json.JSONDecodeError:
        print("Failed to decode JSON response.")
        return None


In [88]:
df_paragraphs["provided_arguments"] = df_paragraphs["paragraph"].apply(extract_arguments_and_sentiment)

In [90]:
df_paragraphs.to_csv("final_arguments_testing.csv")

In [ ]:
import pandas as pd
import ast

# Initialize a list to store the exploded rows
rows = []

# Iterate through the DataFrame and expand the dictionary into rows
for index, row in df_paragraphs_testing.iterrows():
    provided_args = row["provided arguments"]
    # Convert string to dictionary if needed
    if isinstance(provided_args, str):
        try:
            provided_args = ast.literal_eval(provided_args)  # Convert string to dictionary
        except (ValueError, SyntaxError):
            print(f"Invalid format for provided_args at index {index}: {provided_args}")
            continue  # Skip invalid entries

    # Ensure it's a dictionary
    if isinstance(provided_args, dict):
        #print(f"Processed provided_args: {provided_args}")
        for argument, sentiment in provided_args.items():
            rows.append({
                "ID": row["ID"],
                "Argument": argument,
                "Sentiment": sentiment
            })

# Create a new DataFrame from the exploded rows
df_result = pd.DataFrame(rows)

# Display the resulting DataFrame
print(df_result)

     ID             Argument Sentiment
0     1          Solid 1QF20  positive
1     1     Long-term Growth  positive
2     1  Channel Development  positive
3     1     Positive Traffic  positive
4     1   Buying Opportunity  positive
..   ..                  ...       ...
845  20     Market Expansion  positive
846  20        US EV credits  positive
847  20   raw material costs  negative
848  20        US regulation  negative
849  20      Model 3Y demand  negative

[850 rows x 3 columns]


In [ ]:
df_result.to_csv("list_of_arguments3.csv")

In [ ]:
df_result.info()

NameError: name 'df' is not defined

In [ ]:
import pandas as pd
import json
from collections import Counter, defaultdict

# Initialize a dictionary to hold counts for each key and sentiment
sentiment_counts = defaultdict(lambda: Counter({'positive': 0, 'neutral': 0, 'negative': 0}))

# Loop through each entry in the provided_arguments column
for entry in df_paragraphs_testing["provided arguments"]:
    # Ensure the entry is a dictionary
    if isinstance(entry, str):
        try:
            # Convert string to dictionary if it's in JSON-like format
            arguments_dict = json.loads(entry.replace("'", '"'))  # Replace single quotes with double quotes for JSON
        except json.JSONDecodeError:
            continue  # Skip any rows that can't be parsed as dictionaries
    elif isinstance(entry, dict):
        arguments_dict = entry
    else:
        continue  # Skip if entry is not a dictionary or parseable string

    # Update counts for each key and sentiment
    for key, sentiment in arguments_dict.items():
        sentiment_counts[key][sentiment] += 1

# Convert the result to a DataFrame for easier viewing
sentiment_overview_df = pd.DataFrame(sentiment_counts).T
sentiment_overview_df.columns = ["positive", "neutral", "negative"]
sentiment_overview_df.index.name = "Argument"

# Display the overview DataFrame
sentiment_overview_df.reset_index(inplace=True)
sentiment_overview_df.sort_values(by=["positive","negative","neutral"],ascending=False)


,Argument,positive,neutral,negative
5,Financial Guidance,6,2,0
0,Financial Performance,5,2,0
6,Share Repurchase,5,1,0
24,Revenue Growth,5,1,0
2,Capital Management,4,0,0
1,Dividend Policy,3,1,1
3,Earnings Guidance,3,1,0
10,Operational Efficiency,3,1,0
31,Product Strategy,3,1,0
32,Growth Strategy,3,0,0


# Check amount of intensifiers

In [ ]:
import re

# Function to count intensifiers in a single text
def count_intensifiers(text, intensifiers):
    # Convert to lowercase to make it case-insensitive
    words = re.findall(r'\b\w+\b', text.lower())
    return sum(words.count(word) for word in intensifiers)

In [ ]:
# Apply the function to the "Sentence" column of the DataFrame
df_paragraphs_testing["Basic Intensifier"] = df_paragraphs_testing["paragraphs"].apply(count_intensifiers, intensifiers=basic_intensifiers)
df_paragraphs_testing["Strong Intensifier"] = df_paragraphs_testing["paragraphs"].apply(count_intensifiers, intensifiers=strong_intensifiers)
df_paragraphs_testing["Formal Intensifier"] = df_paragraphs_testing["paragraphs"].apply(count_intensifiers, intensifiers=formal_emotive_intensifiers)

/var/folders/2h/tyzkflxs465698k6z8l3ktj00000gn/T/ipykernel_75223/694084076.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_paragraphs_testing["Basic Intensifier"] = df_paragraphs_testing["paragraphs"].apply(count_intensifiers, intensifiers=basic_intensifiers)
/var/folders/2h/tyzkflxs465698k6z8l3ktj00000gn/T/ipykernel_75223/694084076.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_paragraphs_testing["Strong Intensifier"] = df_paragraphs_testing["paragraphs"].apply(count_intensifiers, intensifie

In [ ]:
df_paragraphs_testing

,Unnamed: 0,ID,paragraphs,provided arguments,Basic Intensifier,Strong Intensifier,Formal Intensifier
0,0,1,We maintain our Overweight rating and raise ou...,"{'Financial Performance': 'positive', 'Dividen...",0,0,0
1,1,1,We remain in the minority with less than a thi...,"{'Balance Sheet': 'negative', 'Dividend Policy...",1,0,0
2,2,1,"Adjustments reect Q3 2021 actuals, our updated...","{'Operational Efficiency': 'positive', 'Capita...",0,0,0
3,3,1,XOM expects upstream volumes to increase on lo...,"{'Operational Efficiency': 'neutral', 'Upstrea...",0,0,0
4,4,1,XOM announced a 10bn share repurchase Buybacks...,"{'Share Repurchase': 'positive', 'Dividend Pol...",0,0,0
5,5,1,XOM announced annual capex range of 2025bn goi...,"{'Capital Management': 'positive', 'Cost Struc...",1,1,0
6,6,1,EPS of 1.57 was in betweenour estimate of 1.58...,"{'Financial Performance': 'neutral', 'Operatio...",0,0,0
7,7,1,Our 75 price target is based on 6.5x our 2023 ...,"{'Valuation': 'neutral', 'Earnings Guidance': ...",0,0,0
8,8,1,Systematic risks to our targetrating include c...,"{'Regulatory Environment': 'negative', 'Operat...",0,0,0
9,9,1,XOM We have an Overweight rating on XOM. Opex ...,"{'Financial Performance': 'positive', 'Cost Ma...",0,0,0
